In [1]:
# 폴더 내 파일을 모두 불러옴(생략가능)
import glob

post_path = "C:\\Users\\user\\Desktop\\전처리데이터\\post\\*"
post_file_list = glob.glob(post_path)
post_file_list

['C:\\Users\\user\\Desktop\\전처리데이터\\post\\isipdae.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\isipdae_nam.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\isipdae_yeo.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\samsipdae.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\sasipdae.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\sipdae.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\sipdae_nam.csv',
 'C:\\Users\\user\\Desktop\\전처리데이터\\post\\sipdae_yeo.csv']

In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [82]:
# 파일을 데이터프레임으로 불러옴
import pandas as pd

post_data = pd.read_csv(post_file_list[7])

In [83]:
post_data['crawling_time']=post_data['crawling_time'].str[0:10]+post_data['crawling_time'].str[-14:-5]

In [84]:
from datetime import datetime, timedelta
post_data['crawling_time'] = pd.to_datetime(post_data['crawling_time'], format="%Y-%m-%d %H:%M:%S")

In [50]:
# Python2DB class
import pymysql


class Python2DB():
    def __init__(self):
        self.WL_DB = pymysql.connect(host='220.90.200.176',
                                     port=3306,
                                     user='multi',
                                     passwd='multi!)@(',
                                     db = 'SocialAnalysis',
                                     charset='utf8')
        self.cursor = self.WL_DB.cursor()
        
    def select(self, table, column):
        sql_qr = """
            SELECT {0} FROM {1}
        """.format(column, table)
        self.cursor.execute(sql_qr)
        return self.cursor.fetchall()

    def insert(self, table, columns, values):
        sql_qr2 = """
            INSERT INTO {0}({1})
            VALUES ({2})
        """.format(table, columns, values)
        self.cursor.execute(sql_qr2)
        self.WL_DB.commit()

    def update(self, table, set_content, where_content):
        sql_qr3 = """
            UPDATE {0}
            SET {1}
            WHERE {2}
        """.format(table, set_content, where_content)
        self.cursor.execute(sql_qr3)
        self.WL_DB.commit()

    def delete(self, table, where_content):
        sql_qr4 = """
            DELETE FROM {0} WHERE {1}
        """.format(table, where_content)
        self.cursor.execute(sql_qr4)
        self.WL_DB.commit()

In [86]:
import re

db_class = Python2DB()  # 클래스 객체 선언
post_data_list = []  # 데이터 확인을 위한 리스트 선언
EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)  # 이모티콘 제거를 위한 EMOJI 변수

for i in range(len(post_data)):  # 데이터프레임의 크기만큼 돌게 설정
    
    # 각 데이터 전처리
    if(str(post_data['like_count'][i]) != 'nan'):
        like_count = int(post_data['like_count'][i])
    else:
        like_count = 0
        
    if(str(post_data['view_count'][i]) != 'nan'):  # 리스트로 저장된 여러 미디어의 view_count중 맨 앞 미디어의 view카운트를 저장(기준 view_count만 저장)
        try:
            view_count = int(re.findall('(?<=\[)[1-9]\w+', post_data['view_count'][i])[0])
        except:
            view_count = 0
    else:
        view_count = 0
        
    if(str(post_data['media_url'][i]) != 'nan'):
        media_url = post_data['media_url'][i].replace(',', '  ')
    else:
        media_url = ''
    
    if(str(post_data['content'][i]) != 'nan'):  # 이모티콘 제거 및 전처리 후 1000자까지만 저장
        content = EMOJI.sub(r'', post_data['content'][i]).replace(',', '  ').replace('"', '').replace("'", "")[:1000]
    else:
        content = ''
        
    if(str(post_data['region_tag'][i]) != 'nan'):
        region_tag = EMOJI.sub(r'', post_data['region_tag'][i])
    else:
        region_tag = ''
        
        
    if(str(post_data['hashtag'][i]) != 'nan'): # 이모티콘 제거 및 전처리 후 1000자까지만 저장
        hashtag = EMOJI.sub(r'', post_data['hashtag'][i].replace(' #', '  ')[1:]).replace(',', '  ').replace('"', '').replace("'", "")[:1000]
    else:
        hashtag = ''
        
    post_data_list.append(
        {"inner_id": str(post_data['inner_id'][i]),
         "post_date": post_data['post_date'][i],
         "crawling_time": post_data['crawling_time'][i],
         "like_count": like_count,
         "view_count" : view_count,
         "url" : post_data['url'][i].replace(',', '  '),
         "shortcode" : post_data['shortcode'][i],
         "media_url" : media_url,
         "content" : content,
         "region_tag" : region_tag,
         "hashtag" : hashtag,
         "team_idx" : post_data['team_idx'][i],
        })
    try:
        db_class.insert('POST_1', 'shortcode, inner_id, post_date, crawling_time, like_count, view_count, url, media_url,content, region_tag, hashtag, team_idx',
               '"{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", {}'.format(
                   post_data['shortcode'][i], str(post_data['inner_id'][i]), post_data['post_date'][i], post_data['crawling_time'][i], like_count, view_count, 
                   post_data['url'][i].replace(',', '  '), media_url,
                   content, region_tag, hashtag, post_data['team_idx'][i]
               ))
    except:
        print("error = " + '"{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", {}'.format(
                   post_data['shortcode'][i], str(post_data['inner_id'][i]), post_data['post_date'][i], post_data['crawling_time'][i], like_count, view_count, 
                   post_data['url'][i].replace(',', '  '), media_url,
                   content, region_tag, hashtag, post_data['team_idx'][i], post_data['age'][i]))
        pass

error = "B-r6ttXhJ-W", "11135851667", "2020-04-08 01:23:05", "2020-09-16 19:03:03", "32", "0", "https://www.instagram.com/p/B-r6ttXhJ-W", "https://instagram.ficn6-1.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/92343891_643767966480054_4910339308493487861_n.jpg?_nc_ht=instagram.ficn6-1.fna.fbcdn.net&_nc_cat=106&_nc_ohc=BkjimD_WZlsAX8vH5Yt&_nc_tp=15&oh=d6de8d6e6fbd1601abb8bbfd5b1338a9&oe=5F89F4ED", "신이난당 재미난당 \*^_^*/+_+\", "순천시 연향동", "", 1
error = "4eXBkPK2FZ", "1755412775", "2015-06-28 22:23:30", "2020-09-17 00:29:29", "9", "0", "https://www.instagram.com/p/4eXBkPK2FZ", "https://instagram.ficn6-1.fna.fbcdn.net/v/t51.2885-15/e15/11330603_1098297563518080_358287711_n.jpg?_nc_ht=instagram.ficn6-1.fna.fbcdn.net&_nc_cat=100&_nc_ohc=DIAdt9t7MuwAX-eS0Cj&_nc_tp=18&oh=b6afea59f32876043a3d4404f021a5d3&oe=5F8C4EC0", "#물 위 레스토랑 #시크릿가든 #스테이크 #맛남 #한강에서 #분위기짱짱 #송어파스타 #고르곤졸라 #프로포즈 장소 #야경 #한강데이트 #잠실 #나름 물위에 떠 서 식사하는거 찍고싶고 야경도 찍고 싶고 비춤의 미학을 나타내 보았다 ㅎ", "한강 "시크릿가든" 선상레스토랑", "   시크릿가든   스테이크   맛남   한강에서   분위